In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from dataclasses import dataclass

@dataclass
class ModelConfig:
    d_model: int = 1024
    n_heads: int = 16
    d_ff: int = 2816
    vocab_size: int = 32000
    num_encoder_layers: int = 6
    num_decoder_layers: int = 3
    rope_theta: float = 10000.0
    dropout: float = 0.0

